<a href="https://colab.research.google.com/github/apaleblueman/cap-comp215-personal/blob/main/labs/lab03-classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Gurchetan Singh
#### Date: 27 january 2024

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [117]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [118]:
API_KEY = 'XCTMti69BWbORNZFDrO1LyZOcs8qFlax8ZYcNHL6'

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-27'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

138 Near Earth Objects found for week of 2023-01-27
{'absolute_magnitude_h': 24.96,
 'close_approach_data': [{'close_approach_date': '2023-01-29',
                          'close_approach_date_full': '2023-Jan-29 16:17',
                          'epoch_date_close_approach': 1675009020000,
                          'miss_distance': {'astronomical': '0.0271846797',
                                            'kilometers': '4066770.179752239',
                                            'lunar': '10.5748404033',
                                            'miles': '2526973.8123653382'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '128009.4965423953',
                                                'kilometers_per_second': '35.558193484',
                                                'miles_per_hour': '79540.1230043567'}}],
 'designation': '2023 CF',
 'estimated_diameter': {'feet': {'estimated_diameter_max':

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [119]:
# Ex.1 your code here
class Asteroid:
  def __init__(self,id,name,estimated_diameter_max,estimated_diameter_min,is_potentially_hazardous):
      self.id = id
      self.name = name
      self.estimated_diameter_max = estimated_diameter_max
      self.estimated_diameter_min = estimated_diameter_min
      self.is_potentially_hazardous = is_potentially_hazardous
      self.close_approaches:list = []
  def __str__(self):
      return f"neo name: {self.name}, id: {self.id}, max diameter: {self.estimated_diameter_max}, min diameter: {self.estimated_diameter_min}, dangerous: {self.is_potentially_hazardous}"
  @classmethod
  def from_NEO(cls, neo_id):
        """Returns a NEO object given a neo id (refrence) and includes a list of close approaches of that object"""
        data = get_neo(neo_id)
        neo_object = cls(data['id'], data['designation'], data['estimated_diameter']['meters']['estimated_diameter_max'],data['estimated_diameter']['meters']['estimated_diameter_min'], data['is_potentially_hazardous_asteroid'])
        #used chatGPT to understand how to use attribute assignment instead of using intermediate variables for list comprenhension
        #in instance variable close_approaches list modification
        neo_object.close_approaches = [CloseApproach.fromCOPdata(neo_object, i) for i in range(len(data['close_approach_data']))]

        return neo_object



## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [120]:
# Ex. 2 your code here
asteroid_object_1 = Asteroid.from_NEO('54339874')
assert str(asteroid_object_1) == 'neo name: 2023 BM1, id: 54339874, max diameter: 63.3928064522, min diameter: 28.3501249023, dangerous: False'
str(asteroid_object_1.close_approaches[0]) #testing custom __str__ method defined in Close approach class for a particular CloseApproachObject

"54339874 is a CloaseApproach class object orbiting Earth with velocity of {'kilometers_per_second': '20.4193538659', 'kilometers_per_hour': '73509.6739172087', 'miles_per_hour': '45676.0526626122'} "

## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
        

  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [121]:
class CloseApproach:
    def __init__(self, neo, orbiting_body, approach_date, miss_distance, relative_velocity):
        self.neo = neo
        self.orbiting_body = str(orbiting_body)
        self.approach_date = datetime.datetime.strptime(approach_date, '%Y-%m-%d')
        self.miss_distance = float(miss_distance)
        self.relative_velocity = relative_velocity

    def __str__(self):
      return f"{self.neo} is a CloaseApproach class object orbiting {self.orbiting_body} with velocity of {self.relative_velocity} "
    @classmethod
    def fromCOPdata(cls, neo_obj , index_val):
              """returns a close approach object with attributes given a neo object and index value of a close appraoch data record"""
              neo_data = get_neo(neo_obj.id)['close_approach_data'][index_val]

              return cls(neo_obj.id, neo_data['orbiting_body'],neo_data['close_approach_date'],neo_data['miss_distance']['kilometers'],neo_data['relative_velocity'])


## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [126]:
# Ex. 4 your code here
my_neo_object = Asteroid.from_NEO(neo['id'])


'neo name: 2023 CF, id: 54340713, max diameter: 60.5396601689, min diameter: 27.0741590945, dangerous: False'

## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here